In [2]:
import rebound
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner

### Data Import

In [3]:
data1 = np.genfromtxt('koi707_timing/koi0707.01.tt')
data2 = np.genfromtxt('koi707_timing/koi0707.02.tt')
data3 = np.genfromtxt('koi707_timing/koi0707.03.tt')
data4 = np.genfromtxt('koi707_timing/koi0707.04.tt')
data5 = np.genfromtxt('koi707_timing/koi0707.05.tt')

data = [data1, data2, data3, data4, data5]

In [4]:
y_array = []
y_err_array = []

for i in range(len(data)):
    
    inter = data[i]
    
    y_array.append(inter[:,0]+inter[:,1])
    y_err_array.append(inter[:,2])

In [5]:
## Planet c (index 1) has the smallest initial time

final_y = []
for i in (y_array):
    final_y.append(i - y_array[1][0])
    

In [6]:
const ={
    'G' :  6.67430e10-11, #SI
    'R_b' : 0.0677, #AU
    'R_c' : 0.1189 , #AU
    'R_d' : 0.1662, #AU
    'R_e' : 0.2138, #AU
    'R_f' : 0.2535, #AU
    'P_b' : 5.7, # days
    'P_c' : 13.2, # days
    'P_d' : 21.8, # days
    'P_e' : 31.8, # days
    'P_f' : 41,# days
    'M_b' : 3.68, #Earth
    'M_c' : 0.39, #Earth
    'M_d': 3.91, #Earth
    'M_e' : 5.57, #Earth
    'M_f': 9.6, #Earth
    'AU' : 1.496e11, #m
    'M_star' : 1.29, # solar mass
    'M_sun' : 1.988e30, # kg
    'day_to_sec' : 86400,
    'M_earth' : 5972e24 #kg
}


def start_pos(time, r, period):
    dist = time*(2*np.pi*r)/period
    theta = dist/r
    x = r*np.cos(theta)
    y = r*np.sin(theta)

    x = x/const['AU']
    y = y/const['AU']
    
    
    return x, y, theta


x_b, y_b, theta_b = start_pos(final_y[0][1], const['R_b']*const['AU'], const['P_b']*const['day_to_sec'])
x_d, y_d, theta_d = start_pos(final_y[2][1], const['R_d']*const['AU'], const['P_d']*const['day_to_sec'])
x_e, y_e, theta_e = start_pos(final_y[3][1], const['R_e']*const['AU'], const['P_e']*const['day_to_sec'])
x_f, y_f, theta_f = start_pos(final_y[4][1], const['R_f']*const['AU'], const['P_f']*const['day_to_sec'])


x = np.array([x_b, const['R_c'], x_d, x_e, x_f])
y = np.array([y_b, 0, y_d, y_e, y_f])

In [7]:
def orb_v2(r, p, theta):
    v = 2*np.pi*r/p
    vy = v*np.sin(theta)
    vx = v*np.cos(theta)
    return vy, vx

v2y_b, v2x_b = orb_v2(const['R_b'], const['P_b'], theta_b)
v2y_c, v2x_c = orb_v2(const['R_c'], const['P_c'], 0)
v2y_d, v2x_d = orb_v2(const['R_d'], const['P_d'], theta_d)
v2y_e, v2x_e = orb_v2(const['R_e'], const['P_e'], theta_e)
v2y_f, v2x_f = orb_v2(const['R_f'], const['P_f'], theta_f)

v02_y = [v2y_b, v2y_c, v2y_d, v2y_e, v2y_f]
v02_x = [v2x_b, v2x_c, v2x_d, v2x_e, v2x_f]